# Bridge MLS Data 🌉

In [2]:
#Import libraries
import pandas as pd
import numpy as np
import json
import requests

In [23]:
dataset_id = 'bridgemls'
server_token = '4c71c15af9f486c827e0307b79d0582a'
resource = 'property'

#r = requests.get(f'https://api.bridgedataoutput.com/api/v2/{dataset_id}/{resource}?access_token={server_token}')
r = requests.get('https://api.bridgedataoutput.com/api/v2/dataset_id/listings?access_token=access_token')

In [24]:
r

<Response [403]>

In [13]:
resource

'properties'

# RapidAPI

In [26]:
import requests

url = "https://mashvisor-api.p.rapidapi.com/rental-rates"

querystring = {"state":"CA","source":"airbnb","city":"Los Angeles","zip_code":"90291"}

headers = {
	"X-RapidAPI-Key": "552c8e0a5emsh03eb078a5d59bdap1e9190jsn571dc949c1bc",
	"X-RapidAPI-Host": "mashvisor-api.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'message': 'You are not subscribed to this API.'}


# Webscrape Redfin

In [ ]:
#Links
#page 1
https://www.redfin.com/city/16409/CA/Sacramento?utm_medium=ppc&utm_term=aud-923999260716:kwd-18479583&utm_content=571911945531&utm_campaign=1034079&gclid=Cj0KCQjwoeemBhCfARIsADR2QCtUnYgxvF1rdzy7Wtzu0lM2uALLwk9kDasnq185i-JZK2lPqnnGH5saAqkwEALw_wcB')

#page 2
https://www.redfin.com/city/16409/CA/Sacramento/page-2?utm_medium=ppc&utm_term=aud-923999260716:kwd-18479583&utm_content=571911945531&utm_campaign=1034079&gclid=Cj0KCQjwoeemBhCfARIsADR2QCtUnYgxvF1rdzy7Wtzu0lM2uALLwk9kDasnq185i-JZK2lPqnnGH5saAqkwEALw_wcB

In [88]:
#Request
from bs4 import BeautifulSoup

r = requests.get('https://www.redfin.com/city/16409/CA/Sacramento/page-2?utm_medium=ppc&utm_term=aud-923999260716:kwd-18479583&utm_content=571911945531&utm_campaign=1034079&gclid=Cj0KCQjwoeemBhCfARIsADR2QCtUnYgxvF1rdzy7Wtzu0lM2uALLwk9kDasnq185i-JZK2lPqnnGH5saAqkwEALw_wcB')

In [ ]:
#Second page to scrape
#https://www.redfin.com/city/16409/CA/Sacramento/page-2?utm_medium=ppc&utm_term=aud-923999260716:kwd-18479583&utm_content=571911945531&utm_campaign=1034079&gclid=Cj0KCQjwoeemBhCfARIsADR2QCtUnYgxvF1rdzy7Wtzu0lM2uALLwk9kDasnq185i-JZK2lPqnnGH5saAqkwEALw_wcB

In [89]:
#Parse HTML
soup = BeautifulSoup(r.content, 'html.parser')

In [90]:
#Recieve PRICE OF HOME
prices = []
for idx in range(len(soup.find_all('div', {'class': 'v2 interactive'}))):
    try:
        prices.append(soup.find_all('div', {'class': 'v2 interactive'})[idx].find_all('span', {'class': 'homecardV2Price'})[0].text)
    except:
        pass
#prices

In [91]:
property_stats = []

for idx in range(len(soup.find_all('div', {'class': 'v2 interactive'}))):
    try:
        property_stats.append((soup.find_all('div', {'class': 'v2 interactive'})[idx].find_all('div', {'class': 'stats'})[0].text.split(' ')[0],
                              soup.find_all('div', {'class': 'v2 interactive'})[idx].find_all('div', {'class': 'stats'})[1].text.split(' ')[0],
                              soup.find_all('div', {'class': 'v2 interactive'})[idx].find_all('div', {'class': 'stats'})[2].text.split(' ')[0]))
#         print(soup.find_all('div', {'class': 'v2 interactive'})[idx].find_all('div', {'class': 'stats'})[0].text.split(' ')[0])
#         print(soup.find_all('div', {'class': 'v2 interactive'})[idx].find_all('div', {'class': 'stats'})[1].text.split(' ')[0])
#         print(soup.find_all('div', {'class': 'v2 interactive'})[idx].find_all('div', {'class': 'stats'})[2].text.split(' ')[0])
#         print()
    except:
        pass
#Bed
#bath
#sqft

In [92]:
property_names = soup.find_all('div', {'class': 'link-and-anchor'})

In [93]:
property_df = pd.DataFrame(data=list(zip(property_names, prices, property_stats)), columns=['Property_Name', 'Price', 'Stats'])

In [94]:
#Cleanse
property_df['Beds'] = property_df['Stats'].apply(lambda i: i[0])
property_df['Baths'] = property_df['Stats'].apply(lambda i: i[1])
property_df['Sqft'] = property_df['Stats'].apply(lambda i: i[2])
property_df['Property_Name'] = property_df['Property_Name'].apply(lambda i: i.text)

In [95]:
property_df.drop(columns='Stats', axis=1, inplace=True)
property_df

,Property_Name,Price,Beds,Baths,Sqft
0,"109 River Chase Cir, Sacramento, CA 95864","$728,000",3,2,"1,694"
1,"5520 Bradford Dr, Sacramento, CA 95820","$365,000",3,1,"1,139"
2,"5308 58th St, Sacramento, CA 95820","$320,000",3,1,"1,039"
3,"5622 Hillsdale Blvd Unit A, Sacramento, CA 95842","$210,000",2,1.5,933
4,"520 Santa Ana Ave, Sacramento, CA 95838","$540,000",4,2,"1,100"
5,"9982 Froom Cir, Sacramento, CA 95827","$49,950",2,1,690
6,"749 41st St, Sacramento, CA 95819","$1,649,000",5,3,"3,000"
7,"5451 70th St, Sacramento, CA 95820","$395,000",2,1,858
8,"523 18th St, Sacramento, CA 95811","$1,050,000",5,4,"2,758"
9,"3320 Sendero St, Sacramento, CA 95838","$344,900",2,1,"1,028"


# Final

#####  #Links
#page 1
https://www.redfin.com/city/16409/CA/Sacramento?utm_medium=ppc&utm_term=aud-923999260716:kwd-18479583&utm_content=571911945531&utm_campaign=1034079&gclid=Cj0KCQjwoeemBhCfARIsADR2QCtUnYgxvF1rdzy7Wtzu0lM2uALLwk9kDasnq185i-JZK2lPqnnGH5saAqkwEALw_wcB')

#page 2
https://www.redfin.com/city/16409/CA/Sacramento/page-2?utm_medium=ppc&utm_term=aud-923999260716:kwd-18479583&utm_content=571911945531&utm_campaign=1034079&gclid=Cj0KCQjwoeemBhCfARIsADR2QCtUnYgxvF1rdzy7Wtzu0lM2uALLwk9kDasnq185i-JZK2lPqnnGH5saAqkwEALw_wcB

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas
import numpy

In [27]:
#Collect All Pages of Data for Sacramento
prices = []
property_stats = []
property_names_ls = []
for idx in range(1,10):
    if idx < 2:
        url = 'https://www.redfin.com/city/16409/CA/Sacramento?utm_medium=ppc&utm_term=aud-923999260716:kwd-18479583&utm_content=571911945531&utm_campaign=1034079&gclid=Cj0KCQjwoeemBhCfARIsADR2QCtUnYgxvF1rdzy7Wtzu0lM2uALLwk9kDasnq185i-JZK2lPqnnGH5saAqkwEALw_wcB'
    else:
        url = f'https://www.redfin.com/city/16409/CA/Sacramento/page-{idx}?utm_medium=ppc&utm_term=aud-923999260716:kwd-18479583&utm_content=571911945531&utm_campaign=1034079&gclid=Cj0KCQjwoeemBhCfARIsADR2QCtUnYgxvF1rdzy7Wtzu0lM2uALLwk9kDasnq185i-JZK2lPqnnGH5saAqkwEALw_wcB'
        
    #Parse HTML
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    #Recieve PRICE OF HOME
    for idx1 in range(len(soup.find_all('div', {'class': 'v2 interactive'}))):
        try:
            prices.append(soup.find_all('div', {'class': 'v2 interactive'})[idx1].find_all('span', {'class': 'homecardV2Price'})[0].text)
        except:
            pass
    
    #Gather Property Stats
    for idx2 in range(len(soup.find_all('div', {'class': 'v2 interactive'}))):
        try:
            property_stats.append((soup.find_all('div', {'class': 'v2 interactive'})[idx2].find_all('div', {'class': 'stats'})[0].text.split(' ')[0],
                                  soup.find_all('div', {'class': 'v2 interactive'})[idx2].find_all('div', {'class': 'stats'})[1].text.split(' ')[0],
                                  soup.find_all('div', {'class': 'v2 interactive'})[idx2].find_all('div', {'class': 'stats'})[2].text.split(' ')[0]))
        except:
            pass
    
    property_names = soup.find_all('div', {'class': 'link-and-anchor'})
    property_names_ls = property_names_ls + [x.text for x in property_names]
    
#Convert To DF
property_df = pd.DataFrame(data=list(zip(property_names_ls, prices, property_stats)), columns=['Property_Name', 'Price', 'Stats'])

#Cleanse
property_df['Beds'] = property_df['Stats'].apply(lambda i: i[0])
property_df['Baths'] = property_df['Stats'].apply(lambda i: i[1])
property_df['Sqft'] = property_df['Stats'].apply(lambda i: i[2])

property_df.drop(columns='Stats', axis=1, inplace=True)

In [28]:
property_df

,Property_Name,Price,Beds,Baths,Sqft
0,"3829 Elm St, Sacramento, CA 95838","$279,900",3,1,918
1,"413 Tailoff Ln Unit 91B, Sacramento, CA 95818","$459,900",2,2.5,"1,116"
2,"5321 Dana, Sacramento, CA 95822","$525,000",3,2,"1,192"
3,"4900 Parker Ave, Sacramento, CA 95820","$399,000",4,2,"1,502"
4,"6735 Tortola Way, Sacramento, CA 95828","$309,000",2,2,"1,134"
...,...,...,...,...,...
345,"5360 15th Ave, Sacramento, CA 95820","$645,000",4,2,"1,630"
346,"0 Garden Hwy, Sutter, CA 95837","$257,900",—Beds,—Baths,0.61
347,"150 Del Verde Cir #98, Sacramento, CA 95833","$219,500",2,1,956
348,"3561 N St, Sacramento, CA 95816","$1,299,000",4,2,"1,260"


# Final1
- Updated to scrape `#of pages` for each city (No need for manual input)
- Updated Links so no need for hardcoding - link is established when user inputs city
- **NEXT STEPS:**
- - UPDATE TO GATHER MULTIPLE/OTHER CITIES OF DATA

In [97]:
city = input('Enter City: ')
codes_df = pd.read_excel('./codes_and_cities.xlsx')

code = codes_df[codes_df['City']==city]['Code'].values[0]


url = f'https://www.redfin.com/city/{code}/CA/{city}/page-2?utm_medium=ppc&utm_term=aud-923999260716:kwd-18479583&utm_content=571911945531&utm_campaign=1034079&gclid=Cj0KCQjwoeemBhCfARIsADR2QCtUnYgxvF1rdzy7Wtzu0lM2uALLwk9kDasnq185i-JZK2lPqnnGH5saAqkwEALw_wcB'
#url

#We can fix this in order to validate first letter is capitalized and rest lower

Enter City: Rocklin


In [98]:
#Webscrape to gather Num of Pages on city website
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
controls = soup.find_all('div', {'class' : 'PagingControls'})
n = int(controls[0].find_all('a')[-1].text)
print('Number of Pages:', n) #Num of pages

3

In [63]:
import pandas as pd

In [99]:
#Collect All Pages of Data for city
prices = []
property_stats = []
property_names_ls = []
for idx in range(n, 0, -1):
    print(f'Collecting Page {idx}')
    if idx < 2:
        url1 = url.replace(f'/page-2', '')
    else:
        url1 = url.replace('page-2', f'page-{idx}')
        
    #Parse HTML
    r = requests.get(url1)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    #Recieve PRICE OF HOME
    for idx1 in range(len(soup.find_all('div', {'class': 'v2 interactive'}))):
        try:
            prices.append(soup.find_all('div', {'class': 'v2 interactive'})[idx1].find_all('span', {'class': 'homecardV2Price'})[0].text)
        except:
            pass
    
    #Gather Property Stats
    for idx2 in range(len(soup.find_all('div', {'class': 'v2 interactive'}))):
        try:
            property_stats.append((soup.find_all('div', {'class': 'v2 interactive'})[idx2].find_all('div', {'class': 'stats'})[0].text.split(' ')[0],
                                  soup.find_all('div', {'class': 'v2 interactive'})[idx2].find_all('div', {'class': 'stats'})[1].text.split(' ')[0],
                                  soup.find_all('div', {'class': 'v2 interactive'})[idx2].find_all('div', {'class': 'stats'})[2].text.split(' ')[0]))
        except:
            pass
    
    property_names = soup.find_all('div', {'class': 'link-and-anchor'})
    property_names_ls = property_names_ls + [x.text for x in property_names]
    
#Convert To DF
property_df = pd.DataFrame(data=list(zip(property_names_ls, prices, property_stats)), columns=['Property_Name', 'Price', 'Stats'])

#Cleanse
property_df['Beds'] = property_df['Stats'].apply(lambda i: i[0])
property_df['Baths'] = property_df['Stats'].apply(lambda i: i[1])
property_df['Sqft'] = property_df['Stats'].apply(lambda i: i[2])

property_df.drop(columns='Stats', axis=1, inplace=True)

In [100]:
property_df

,Property_Name,Price,Beds,Baths,Sqft
0,"5691 Brookshire Ct, Rocklin, CA 95677","$1,008,930",4,3,"3,287"
1,"6812 Camborne Way, Rocklin, CA 95677","$985,000",7,4,"3,656"
2,"4139 Peters Way, Rocklin, CA 95765","$1,069,990",4,3,"2,243"
3,"3962 Rawhide Rd, Rocklin, CA 95677","$399,999",—Beds,—Baths,0.77
4,"1222 Buck Ridge Dr, Rocklin, CA 95765","$1,119,990",4,3,"2,741"
...,...,...,...,...,...
110,"3422 Cook St, Rocklin, CA 95765","$590,000",4,2.5,"1,900"
111,"6884 Cherry Ridge Cir, Roseville, CA 95678","$899,900",5,3,"3,121"
112,"3408 Stoney Rd, Rocklin, CA 95765","$549,000",3,2,"1,577"
113,"4230 Midas Ave, Rocklin, CA 95677","$689,000",5,2,"1,950"


# Final2 
### Gather property data for houses in applicable cities - From User Input
#### Applicable cities:
- `Rocklin`
- `Lincoln`
- `North Highlands`
- `Roseville`

In [152]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import os

In [153]:
#Cities to View Property Data - ENTER HERE (This can be Improved via GUI, etc.)
cities = ['Sacramento', 'Lincoln', 'Rocklin']

#Read in URL codes
codes_df = pd.read_excel('./codes_and_cities.xlsx')

In [154]:
#Code to collect Property Data city-by-city
for city in cities:
    print('\nCollecting {} Data...'.format(city))
    
    #Set URL Path destination
    code = codes_df[codes_df['City']==city]['Code'].values[0]
    url = f'https://www.redfin.com/city/{code}/CA/{city}/page-2?utm_medium=ppc&utm_term=aud-923999260716:kwd-18479583&utm_content=571911945531&utm_campaign=1034079&gclid=Cj0KCQjwoeemBhCfARIsADR2QCtUnYgxvF1rdzy7Wtzu0lM2uALLwk9kDasnq185i-JZK2lPqnnGH5saAqkwEALw_wcB'
    
    #Webscrape to gather Num of Pages on city website
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    controls = soup.find_all('div', {'class' : 'PagingControls'})
    n = int(controls[0].find_all('a')[-1].text)
    print('Number of Pages:', n) #Num of pages
    
    prices = []
    property_stats = []
    property_names_ls = []
    for idx in range(n, 0, -1):
        print(f'Collecting Page {idx}')
        if idx < 2:
            url1 = url.replace(f'/page-2', '')
        else:
            url1 = url.replace('page-2', f'page-{idx}')

        #Parse HTML
        r = requests.get(url1)
        soup = BeautifulSoup(r.content, 'html.parser')

        #Recieve PRICE OF HOME
        for idx1 in range(len(soup.find_all('div', {'class': 'v2 interactive'}))):
            try:
                prices.append(soup.find_all('div', {'class': 'v2 interactive'})[idx1].find_all('span', {'class': 'homecardV2Price'})[0].text)
            except:
                pass

        #Gather Property Stats
        for idx2 in range(len(soup.find_all('div', {'class': 'v2 interactive'}))):
            try:
                property_stats.append((soup.find_all('div', {'class': 'v2 interactive'})[idx2].find_all('div', {'class': 'stats'})[0].text.split(' ')[0],
                                      soup.find_all('div', {'class': 'v2 interactive'})[idx2].find_all('div', {'class': 'stats'})[1].text.split(' ')[0],
                                      soup.find_all('div', {'class': 'v2 interactive'})[idx2].find_all('div', {'class': 'stats'})[2].text.split(' ')[0]))
            except:
                pass

        property_names = soup.find_all('div', {'class': 'link-and-anchor'})
        property_names_ls = property_names_ls + [x.text for x in property_names]

    #Convert To DF
    property_df = pd.DataFrame(data=list(zip(property_names_ls, prices, property_stats)), columns=['Property_Name', 'Price', 'Stats'])

    #Cleanse
    property_df['Beds'] = property_df['Stats'].apply(lambda i: i[0])
    property_df['Baths'] = property_df['Stats'].apply(lambda i: i[1])
    property_df['Sqft'] = property_df['Stats'].apply(lambda i: i[2])

    property_df.drop(columns='Stats', axis=1, inplace=True)
    property_df.to_excel(f'{city}_data.xlsx') #Timestamp?


Number of Pages: 9

Number of Pages: 5

Number of Pages: 3


In [166]:
contents = os.listdir(os.getcwd())  #Filter to xlsx city data
dfs = []  

for city in cities:
    print('City: {}'.format(city))
    matching_files = [c for c in contents if city in c and c.endswith('.xlsx')]
    if matching_files:
        file_path = matching_files[0]  
        df = pd.read_excel(file_path)
        print('Length:', len(df), '\n')
        dfs.append(df)

# Concatenate all DataFrames in the list
properties_df = pd.concat(dfs, ignore_index=True)

print('{} Datapoints'.format(len(properties_df)))
properties_df.tail(40)
properties_df = properties_df.iloc[:,1:] 
properties_df.tail(55)

City: Sacramento
Length: 350 

City: Lincoln
Length: 198 

City: Rocklin
Length: 115 

663 Datapoints


,Property_Name,Price,Beds,Baths,Sqft
608,"3946 Rawhide Rd, Rocklin, CA 95677","$1,000,000",4,3,"2,852"
609,"4090 Clubview Ct, Rocklin, CA 95765","$1,695,000",7,5,"5,304"
610,"2346 Clubhouse Dr, Rocklin, CA 95765","$1,550,000",4,4.5,"5,023"
611,"3319 Stanford Village Ct, Rocklin, CA 95765","$634,900",3,2.5,"1,976"
612,"5660 Brookshire Dr, Rocklin, CA 95677","$1,116,428",5,4.5,"3,309"
613,"1232 Buck Ridge Ct, Rocklin, CA 95765","$1,099,990",3,3.5,"2,405"
614,"3707 Stone Temple Ct, Rocklin, CA 95765","$1,398,000",4,3.5,"4,162"
615,"4421 Winners Cir, Rocklin, CA 95677","$519,000",3,2,"1,161"
616,"5344 Humboldt Dr, Rocklin, CA 95765","$735,000",4,2,"2,070"
617,"3804 Calverhall Way, Rocklin, CA 95677","$1,250,000",6,4,"3,803"


In [158]:
# Word to search for


Empty DataFrame
Columns: [Unnamed: 0, Property_Name, Price, Beds, Baths, Sqft]
Index: []
